## Сбор и подготовка данных

In [1]:
import pandas as pd
import os
import torch
from transformers import GPT2Tokenizer
from transformers import pipeline
from torch.utils.data import DataLoader
import sys
sys.path.append(os.path.abspath('src'))

/media/k305-1/LEGEND_960_2/python/ac/venv2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### очистка текста

In [ ]:
import preprocess
prepare_dataset()
split_dataset()

train: 1277164, val: 159646, test: 159646


### токенизация, разбиение на тренировачный, валидационный и тестовый датасеты

In [2]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast
from tqdm import tqdm
from next_token_dataset import *

# Загружаем BERT токенизатор
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


train_texts = pd.read_csv("data/train.csv")
train_texts=train_texts['text'].tolist() 
val_texts = pd.read_csv("data/val.csv")
val_texts = val_texts['text'].tolist() 
# тренировочный и валидационный датасеты
train_dataset = NextTokenDataset(train_texts, tokenizer, max_length=20)
val_dataset = NextTokenDataset(val_texts, tokenizer, max_length=20)

# даталоадеры
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=lambda batch: collate_fn_pad_sequence(batch, tokenizer))
val_loader = DataLoader(val_dataset, batch_size=64, collate_fn=lambda batch: collate_fn_pad_sequence(batch, tokenizer))


## объявление модели

In [4]:
from lstm_train import *


Объявлена модель на основе архитектуры LSTM со следующими параметрами:
embed_dim=128, hidden_dim=256, num_layers=2, dropout=0.3

## обучение LSTM модели

In [ ]:
vocab_size = tokenizer.vocab_size

# объявляем модель
model = LSTMTextGenerator(
    vocab_size=vocab_size,
    embed_dim=128,
    hidden_dim=256,  
    num_layers=2,
    dropout=0.3
)

# Обучение
train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    tokenizer=tokenizer,
    epochs=10,
    lr=0.001,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Сохранение
save_checkpoint(
    model=model,
    optimizer=optim.Adam(model.parameters(), lr=0.001),
    epoch=10,
    train_loss=train_losses[-1],
    val_loss=val_losses[-1],
    filename='lstm_text_generator_final.pt'
)

Модель обучена в течение 10 эпох. Финальные метрики обучения:  Train Loss: 5.3009, Val Loss: 5.1672

## запуск LSTM-модели на тестовых данных

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader
from eval_lstm import *
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer.pad_token = tokenizer.eos_token  


# загрузка модели

model, checkpoint = load_model(
    checkpoint_path='lstm_text_generator_final.pt',
    vocab_size=30522,  # BERT base uncased vocabulary size
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# создаем test dataset
test_texts = pd.read_csv("data/test.csv")['text'].tolist()


test_dataset = NextTokenDataset(test_texts, tokenizer, max_length=20)

print("\n1. Автодополнение...")
token_results, gen_texts, ref_texts = evaluate_model(
    model=model,
    test_dataset=test_dataset,
    tokenizer=tokenizer,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    batch_size=32
)

print_evaluation_results(token_results, "Результаты предсказания текста")

print("\nПримеры:")
for i in range(min(5, len(gen_texts))):
    print(f"Reference: '{ref_texts[i]}'")
    print(f"Generated: '{gen_texts[i]}'")
    print()

модель загружена lstm_text_generator_final.pt
Epoch: 10
Train Loss: 5.2689
Val Loss: 5.1347

1. Автодополнение...
Evaluating model on test dataset...
Total batches: 63638


Evaluating:   2%|▏         | 1049/63638 [00:03<03:53, 268.54it/s]

Processed 1000 batches, current avg ROUGE-1: 0.2206


Evaluating:   3%|▎         | 2035/63638 [00:07<03:48, 269.29it/s]

Processed 2000 batches, current avg ROUGE-1: 0.2224


...

Processed 62000 batches, current avg ROUGE-1: 0.2209


Evaluating:  99%|█████████▉| 63052/63638 [03:54<00:02, 249.74it/s]

Processed 63000 batches, current avg ROUGE-1: 0.2209


Evaluating: 100%|██████████| 63638/63638 [03:56<00:00, 268.91it/s]


          Результаты предсказания текста          
ROUGE-1: 0.2209 (±0.4148)
ROUGE-2: 0.0000 (±0.0000)
Number of samples: 2036384

Примеры:
Reference: 'i'
Generated: 'i'

Reference: 'm'
Generated: 'm'

Reference: 'hell'
Generated: 'not'

Reference: '##a'
Generated: '##a'

Reference: 'bored'
Generated: 'tired'



### примеры предсказаний

In [10]:
sample_prompts = ["well i m hella", "i am so excited", "once upon"]
print("\nПримеры:")
for prompt in sample_prompts:
    generated = model.generate(tokenizer, prompt, max_length=20, device='cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Prompt: '{prompt}' -> '{generated}'")
print()


Примеры:
Prompt: 'well i m hella' -> 'well i m hella i m so sad i can t wait to see him and i'
Prompt: 'i am so excited' -> 'i am so excited i m so sad i can t wait to see you and i m'
Prompt: 'once upon' -> 'once upon the one i can t do it i m so sad to be the best of'



## Запуск предобученного трансформера на тестовых данных

In [3]:
from eval_transformer_pipeline import *
#  test датасет
test_texts = pd.read_csv("data/test.csv")['text'].tolist()
print(f"Loaded {len(test_texts)} test texts")



gpt_results, gpt_examples = evaluate_distilgpt(
        model_name="distilgpt2",
        test_texts=test_texts,
        prompt_length=5,
        max_new_tokens=30,
        batch_size=8,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )


print("Метрики DistilGPT2")
print(f"ROUGE-1: {gpt_results['rouge1']:.4f} (±{gpt_results['rouge1_std']:.4f})")
print(f"ROUGE-2: {gpt_results['rouge2']:.4f} (±{gpt_results['rouge2_std']:.4f})")



Loaded 159646 test texts
Loading distilgpt2...


Generating:   6%|▋         | 10004/159646 [07:44<1:55:09, 21.66it/s]


Processed 10000 samples, Avg ROUGE-1: 0.0655, Avg ROUGE-2: 0.0042


Generating:  13%|█▎        | 20004/159646 [15:24<1:41:24, 22.95it/s]


Processed 20000 samples, Avg ROUGE-1: 0.0656, Avg ROUGE-2: 0.0043


...


Processed 130000 samples, Avg ROUGE-1: 0.0652, Avg ROUGE-2: 0.0040


Generating:  88%|████████▊ | 140004/159646 [1:47:45<15:46, 20.76it/s]


Processed 140000 samples, Avg ROUGE-1: 0.0654, Avg ROUGE-2: 0.0040


Generating: 100%|██████████| 159646/159646 [2:02:49<00:00, 21.66it/s]


Метрики DistilGPT2
ROUGE-1: 0.0652 (±0.0773)
ROUGE-2: 0.0039 (±0.0209)


### примеры предсказаний

In [12]:
print("Примеры")
for i, ex in enumerate(gpt_examples[:5]):
    print(f"\nExample {i+1}:")
    print(f"Prompt: '{ex['prompt']}'")
    print(f"Reference: '{ex['reference']}'")
    print(f"Generated: '{ex['generated']}'")
    print(f"ROUGE-1: {ex['rouge1']:.4f}")

Примеры

Example 1:
Prompt: 'well i m hella'
Reference: ' bored here and i hate that school is just a week away'
Generated: '.
I have to admit I am quite a bit of a fan of the game but the overall experience was fantastic. The game is as good as'
ROUGE-1: 0.1538

Example 2:
Prompt: 'is so excited as c'
Reference: 'g has the package now i have to wait until she comes on msn'
Generated: 'uz you've got to make the most of it.


-
-
-
-
-
-
-
-'
ROUGE-1: 0.1667

Example 3:
Prompt: 'just went shopping but i'
Reference: ' m still bummed about no wrestling for me no triple h'
Generated: ''m not sure how to keep my money on the exchanges for the entire month! Thanks a lot for your support!'
ROUGE-1: 0.1250

Example 4:
Prompt: 'wish i could by'
Reference: ' music from itunes in brazil we re not allowed here you know'
Generated: 'the way i am now in the UK.


It was pretty much the only thing I can remember about the phone's firmware. I remember'
ROUGE-1: 0.0541

Example 5:
Prompt: 'listening to

## Сравнение моделей


Метрики LSTM
ROUGE-1: 0.2209 (±0.4148)
ROUGE-2: 0.0000 (±0.0000)

Примеры предсказаний:
Prompt1: 'well i m hella' 
Generated: 'well i m hella i m so sad i can t wait to see him and i'

Prompt2: 'i am so excited'
Generated: 'i am so excited i m so sad i can t wait to see you and i m'

Prompt3: 'once upon' 
Generated: 'once upon the one i can t do it i m so sad to be the best of'


Метрики DistilGPT2
ROUGE-1: 0.0655 (±0.0776)
ROUGE-2: 0.0039 (±0.0210)


Prompt1: 'well i m hella'
Generated: 'I have to admit I am quite a bit of a fan of the game but the overall experience was fantastic. The game is as good as'

Prompt2: 'is so excited as c'
Generated: 'uz you've got to make the most of it.'

Prompt3: 'just went shopping but i'
Generated: ''m not sure how to keep my money on the exchanges for the entire month! Thanks a lot for your support!'

Prompt4: 'wish i could by'
Generated: 'the way i am now in the UK. It was pretty much the only thing I can remember about the phone's firmware. I remember'

Вывод: обученная нами RNN-модель на основе LSTM генерирует связные, но бессмыссленные фрагменты текста. Совпадение биграмм (ROUGE-2) 0. По всей видимости, требуется дальнейшая оптимизация модели и тренировочных датасетов.  Модель DistilGPT2 в некоторых случаях тоже генерирует бессмысленные повторяющиеся слова, однако на большинство запросов генерирует логичные фрагменты текста, вполне осмысленные в контексте промта. При этом метрики ROUGE имеют очень малые значения, однако по всей видимости это отражает то, что модель была обучена на других данных и способна генерировать принципиально другой ответ, отличный от имеющегося в тестовом датасете, что и отражается на значениях метрик.

Время инференса DistilGPT2 на тестовом датасете 2:02:49
Время инференса LSTM на тестовом датасете 03:56
Таким образом, LSTM более чем 30 раз быстрее DistilGPT2, что также важно при запуске в условиях ограниченных ресурсов/времени.


